# Finance News
from yahoo finance: https://finance.yahoo.com/topic/latest-news/


based on these works
- https://medium.com/@georgemichaeldagogomaynard/web-scraping-dynamic-sites-with-both-selenium-and-beautiful-soup-dim-c679743018de

In [4]:
# Dependencies
# Selenium modules for controlling Chrome browser
from selenium.webdriver import Chrome  # For initializing and controlling the Chrome browser
from selenium import webdriver  # Provides access to the webdriver, allowing interaction with web browsers
from selenium.webdriver.chrome.options import Options  # For configuring Chrome browser options (e.g., headless mode)
from selenium.webdriver.chrome.service import Service  # For managing the ChromeDriver service (e.g., starting, stopping)

# Selenium modules for interacting with web elements
from selenium.webdriver.common.by import By  # For locating elements on a webpage (e.g., By.ID, By.XPATH)
from selenium.webdriver.support.ui import Select  # For interacting with <select> HTML elements (dropdowns)
from selenium.webdriver.support.ui import WebDriverWait  # For implementing explicit waits until a condition is met
from selenium.webdriver.support import expected_conditions as EC  # For defining conditions to wait for (e.g., element visibility)

# Other useful libraries
from fake_useragent import UserAgent  # For generating random user agents to mimic different browsers
import time  # For adding delays (e.g., time.sleep) during the script execution
import requests  # For making HTTP requests to interact with websites directly without using a browser
from bs4 import BeautifulSoup  # For parsing and extracting data from HTML content
import pandas as pd  # For data manipulation, analysis, and creating DataFrames

## Experimenting with Scraper.

In [ ]:
# Testing the scraper (replace print with storing data in a structured format later)
# TODO: make this shit works (scrape ticker from inside article)
import re

news_uri = "https://finance.yahoo.com/topic/latest-news/"

# Use selenium for "dynamic" content loading 
options=Options()
options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
# Block images to speed up loading
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
options.page_load_strategy = 'eager'  

# Add headers or some bs to mimic a browser request to avoid 429 (Too Many Requests)
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(options=options)

try: 
    driver.get(news_uri)
    time.sleep(5)  # Wait for the page to load completely
    
    # Scroll to the bottom to load more news
    scroll_count = 3
    for i in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Wait for new content to load after scrolling
        print (f"Scrolled {i+1}/{scroll_count} times")
    page = driver.page_source
    news_soup = BeautifulSoup(page, 'html.parser')

    #print (page.status_code)  # should be 200 if successful
    #print (page.content[:100])  


    news_list = news_soup.find('ul', class_= "stream-items yf-9xydx9")
    news_listings = news_list.find_all('li', class_= "stream-item story-item yf-9xydx9")
    print (f"Found {len(news_listings)} news articles")

    # TODO: catch None conditions, fix duplicates issue, add ticker percentage change
    for news in news_listings:
        # Want to scrape: title, link, publisher, tickers 
        print(news.find('h3').text)  # Article's title
        
        # Use regex to match 'publishing' followed by any other class names
        print(news.find('div', class_=re.compile(r'publishing.*')).text.split('•')[0]) # Publisher name
        
        '''
        # Extract tickers and their percentage changes
        if news.find('div', class_='taxonomy-links'):  
            tickers = []
            for ticker in news.find('div', class_='taxonomy-links').find_all('span', class_=re.compile(r'ticker-wrapper.*')):
                # get data from the fin-streamer element
                streamer = ticker.find('fin-streamer', {'data-field': 'regularMarketChangePercent'})
                if streamer:
                    print (streamer)
                    symbol = streamer.get('data-symbol')
                    change = streamer.get_text(strip=True) # TODO: fix this shit only loading for the first news (maybe increase wait time or scroll more)
                    tickers.append({'symbol': symbol, 'change': change})
                else:
                    # Fallback if streamer is not found
                    tickers.append({'symbol': ticker.text.strip(), 'change': None})
                
            #print (', '.join(tickers))  # All tickers in one line
            print (tickers)
        else:
            print("No tickers found")
        '''
        print (news.find('a')['href']) # link to the article
        
        #bug is after here?
        #TODO: make this process faster
        news_detail_url = news.find('a')['href']
        
        '''
        news_detail_page = requests.get(news_detail_url, headers=headers)
        news_detail_soup = BeautifulSoup(news_detail_page.content, 'html.parser')
        '''
        # Migrate to selenium
        driver.get(news_detail_url)
        try:
            WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, "section[class*='ticker-list yf-1sa9i6j']")))
        except:
            pass 
            
        news_detail_soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Start looking inside each article!
        # writers, time published, article content, 
        # TODO: might use ticker-wrapper from <div class= "slide"> instead
        print(news_detail_soup.find('div', class_=re.compile(r'byline-attr-author.*')).text.strip()) # Authors
        print(news_detail_soup.find('time').text) # Time published
        
        # bug is caused before here?
        
        # Stock tickers from div stock-story-tickers and remaining-stocks-container (or section ticker-list)
        #print(news_detail_soup.find('section', class_=re.compile(r'ticker-list.*')  ))
        stock_tickers_div = news_detail_soup.find('div', class_=re.compile(r'stock-story-tickers.*'))
        #print (stock_tickers_div)
        tickers = []
        if stock_tickers_div:
            for ticker in stock_tickers_div.find_all('div', class_='slide'):
                streamer = ticker.find('fin-streamer', {'data-field': 'regularMarketChangePercent'})
                if streamer:
                    symbol = streamer.get('data-symbol')    
                    change = streamer.get_text(strip=True) 
                    tickers.append({'symbol': symbol, 'change': change})
        
        print (tickers)
        remaining_stocks_div = news_detail_soup.find('div', class_=re.compile(r'remaining-stocks-container.*'))
        
        '''      
        # Body: for each paragraph in the article content
        body_wrapper = news_detail_soup.find('div', class_='bodyItems-wrapper')
        if body_wrapper:
            for para in body_wrapper.find_all('p'):
                # Check if the paragraph is inside 'read-more-wrapper' to avoid duplication
                if para.find_parent('div', class_='read-more-wrapper'): # if parent is read-more-wrapper
                    continue
                print(para.text.strip())
                
        # 'Read more' section
        read_more_wrapper = news_detail_soup.find('div', class_='read-more-wrapper')
        if read_more_wrapper:
            for para in read_more_wrapper.find_all('p'): 
                print(para.text.strip())
        '''
        
        print ("\n")
except Exception as e:
    print(e)

finally:
    driver.quit()

    
# ul and li(class = story-item) tags are used for listing news articles
# For the content of each news article: <div class=content> ->...(below)..
# inside div(class = footer) ->  <div(class = publishing)> for publisher name and time (1st & 2nd text)
# inside div(class = footer) -> <div(class = taxonomy-links)> -> <div(class=ticker-wrapper)> for each ticker

Scrolled 1/3 times
Scrolled 2/3 times
Scrolled 3/3 times
Found 70 news articles
Palantir uses the '5 Whys' approach to problem solving — here's how it works
Business Insider 
https://finance.yahoo.com/news/palantir-uses-5-whys-approach-110103609.html
Brent D. Griffiths
Sat, November 29, 2025 at 6:01 PM GMT+7
[{'symbol': 'PLTR', 'change': '+1.62%'}]


HELOC rates today, November 29, 2025: Rates fall as holiday cash needs rise
Yahoo Personal Finance 
https://finance.yahoo.com/personal-finance/mortgages/article/heloc-rates-today-saturday-november-29-2025-110050848.html
Hal Bundrick, CFP®  · Senior Writer   Laura Grace Tarpley  · Lead Editor and Content Strategist, Mortgages
Sat, November 29, 2025 at 6:00 PM GMT+7
[]


Mortgage and refinance interest rates today, November 29, 2025: Could the next move be below 6%?
Yahoo Personal Finance 
https://finance.yahoo.com/personal-finance/mortgages/article/mortgage-refinance-interest-rates-today-saturday-november-29-2025-110028470.html
Hal Bundrick

KeyboardInterrupt: 

In [23]:
soup.prettify()[:200] 

'<!DOCTYPE html>\n<html class="smartphone neo-green dock-upscale" data-color-scheme="auto" data-color-theme-enabled="true" lang="en-US" theme="auto">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="g'

In [5]:
# Connect to MongoDB
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv

load_dotenv()
db_username = os.getenv("db_username")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")

if not all([db_username, db_password, db_host]):
    print("Please set db_username, db_password, and db_host in your .env file")
else:
    uri = f"mongodb+srv://{db_username}:{db_password}@{db_host}"
    client = MongoClient(uri, server_api=ServerApi('1'))

    try:
        client.admin.command('ping')
        print("Pinged your deployment. You successfully connected to MongoDB!")
        
        # Create/Select database and collection
        db = client["finance_news_db"] 
        collection = db["numerous_articles"]
        print(f"Connected to database: {db.name}, collection: {collection.name}")
        
    except Exception as e:
        print(e)

Pinged your deployment. You successfully connected to MongoDB!
Connected to database: finance_news_db, collection: numerous_articles


# Real Scraper w/connection to MongoDB

In [ ]:
# Working scraper!
import re

news_uri = "https://finance.yahoo.com/topic/latest-news/"

# We use Selenium because the 'fin-streamer' percentages are loaded via JavaScript
options = Options()
options.add_argument("--headless") 
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(options=options)

# Headers for the requests library
headers = {'User-Agent': userAgent}

try:
    print("Fetching news list via Selenium...")
    driver.get(news_uri)
    time.sleep(3) # Wait for page to load
    
    # Scroll until no new content loads
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_count = 0
    while True:
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait for page to load
        time.sleep(3) 
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            print("Reached the end of the news feed.")
            break
            
        last_height = new_height
        scroll_count += 1
        print(f"Scrolled {scroll_count} times...")
        
    ''' # Old code for scrolling (n times)
    # Scroll to load more news
    scroll_count = 500
    for i in range(scroll_count):
        
        # Scroll incrementally to ensure elements trigger their data load
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(2)
         # THIS SHIT DOESN'T WORK
        
        # "Scroll indefinitely (until end of page)"
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3) # Wait for new items to load
        print(f"Scrolled {i+1}/{scroll_count}")
    ''' 
    page_source = driver.page_source
finally:
    driver.quit()

news_soup = BeautifulSoup(page_source, 'html.parser')

news_list = news_soup.find('ul', class_= "stream-items yf-9xydx9")
news_listings = news_list.find_all('li', class_= "stream-item story-item yf-9xydx9")
print(f"Found {len(news_listings)} articles.")

for news in news_listings:
    news_data = {}
    # From the main listing page: want to scrape title, link, publisher, tickers(???)
    # Title
    title = news.find('h3')
    if not title: continue # Skip if malformed
    news_data['title'] = title.text
    
    # Publisher
    publisher = news.find('div', class_=re.compile(r'publishing.*'))
    news_data['publisher'] = publisher.text.split('•')[0] if publisher else "Unknown"
    
    # Tickers (static)
    tickers = []
    if news.find('div', class_='taxonomy-links'):  
        for ticker in news.find('div', class_='taxonomy-links').find_all('span', class_=re.compile(r'ticker-wrapper.*')):
            
            streamer = ticker.find('fin-streamer', {'data-field': 'regularMarketChangePercent'})
            if streamer:
                symbol = streamer.get('data-symbol')
                change = streamer.get_text(strip=True)
                
                # Fallback to static if text is empty
                if not change:
                    change = None
                tickers.append({'symbol': symbol, 'change': change})
            else:
                tickers.append({'symbol': ticker.text.strip(), 'change': None})     
    news_data['tickers'] = tickers
    
    # Link
    link = news.find('a')['href']
    if not link: continue # If link is missing, skip this article
    news_data['link'] = link
    
    # Fetch Article Details 
    try:
        news_detail_url = news_data['link']
        news_detail_page = requests.get(news_detail_url, headers=headers)
        news_detail_soup = BeautifulSoup(news_detail_page.content, 'html.parser')
        
        # Author
        author = news_detail_soup.find('div', class_=re.compile(r'byline-attr-author.*'))
        news_data['authors'] = author.text.strip() if author else "Unknown"

        # Time
        time = news_detail_soup.find('time')
        news_data['time_published'] = time.text if time else "Unknown"
        
        # Content
        content_text = []
        body_wrapper = news_detail_soup.find('div', class_='bodyItems-wrapper')
        seen_texts = set()
        
        if body_wrapper:
            for para in body_wrapper.find_all('p'):
                text = para.text.strip()
                if not text or text in seen_texts: continue
                if "Most Read from" in text or "Recommended Stories" in text: break
                if para.find_parent('div', class_='read-more-wrapper'): continue
                
                content_text.append(text)
                seen_texts.add(text)
                
        news_data['content'] = "\n".join(content_text)
        
        # Insert into MongoDB
        if 'collection' in globals():
            if collection.count_documents({'link': news_data['link']}) == 0:
                collection.insert_one(news_data)
                print(f"Inserted: {news_data['title']}")
            else:
                print(f"Skipped: {news_data['title']}")
        else:
            print(f"Scraped (DB not connected): {news_data['title']}")
            
    except Exception as e:
        print(f"Error scraping details for {news_data['title']}: {e}")
    

Fetching news list via Selenium...
Scrolled 1 times...
Scrolled 2 times...
Scrolled 3 times...
Scrolled 4 times...
Scrolled 5 times...
Scrolled 6 times...
Scrolled 7 times...
Reached the end of the news feed.
Found 170 articles.
Inserted: 10 truths about the stock market


Skipped: Rising star catch-up: Chad Tredway is back at JPMorgan and at the helm of its $79 billion real estate portfolio


Inserted: Chicago Scraps Flights as Major Storm Brings Snow to Midwest


Skipped: Want Steady Income in Retirement? These Overlooked Tools Can Help and May Make All the Difference


Skipped: The hottest new AI company is…Google?


Skipped: Medicare Advantage woos seniors with plan perks, but a study found they often go unused


Skipped: Why Waiting for a Housing Crash Could Be Costing You Money


Skipped: Early 401(k) Withdrawals Could Cost You $100K—Here’s How to Protect Your Retirement


Inserted: Despite flak for doom-spending their money, Gen Z may be more prepared for retirement than baby bo